charger le dataset

In [1]:
import pandas as pd

# Load the data from the CSV file
df = pd.read_csv('./satellite_S1_new.csv')

In [2]:
from shapely.geometry import Polygon
import ast

def calculate_centroid(geometry):
    # Convert the string representation of the list to an actual list
    geometry_list = ast.literal_eval(geometry)
    # Create a Polygon object
    polygon = Polygon(geometry_list[0])
    # Calculate the centroid
    centroid = polygon.centroid
    return centroid.y, centroid.x

# Apply the function to the 'geometry' column
df['latitude_centroid'], df['longitude_centroid'] = zip(*df['geometry'].apply(calculate_centroid))

print(df[['geometry', 'latitude_centroid', 'longitude_centroid']].head())

                                            geometry  latitude_centroid  \
0  [[[36.801563, -17.829673], [36.802546, -17.875...         -17.852289   
1  [[[30.079766, -1.947669], [30.079851, -1.99392...          -1.970753   
2  [[[-1.734277, 6.731243], [-1.734397, 6.684938]...           6.708030   
3  [[[146.754028, -19.184316], [146.753959, -19.2...         -19.207482   
4  [[[48.748582, 30.637319], [48.749651, 30.59115...          30.614695   

   longitude_centroid  
0           36.826162  
1           30.102794  
2           -1.711179  
3          146.778347  
4           48.775806  


In [4]:
import requests

def get_weather_visualcrossing(api_key, lat, lon, date):
    """
    Récupère les données météorologiques historiques pour une localisation et une date donnée
    via l'API Visual Crossing.

    Args:
        api_key (str): Votre clé API Visual Crossing.
        lat (float): Latitude de la région.
        lon (float): Longitude de la région.
        date (str): Date au format 'YYYY-MM-DD'.

    Returns:
        dict: Données météorologiques ou un message d'erreur.
    """
    base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
    url = f"{base_url}{lat},{lon}/{date}/{date}"
    params = {
        "unitGroup": "metric",  # Unités métriques (Celsius, mm, etc.)
        "key": api_key,
        "include": "obs"  # Inclure les observations historiques
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": response.text}

# convertis ces données géospatiales en longitude et latitude correspondante [[[36.801563, -17.829673], [36.802546, -17.875849], [36.850768, -17.874901], [36.849773, -17.828728], [36.801563, -17.829673]]]

# Exemple d'utilisation
api_key = "PUTUNBTHW5R3Q9K2WUW6MPSD6"  # Remplacez par votre clé API Visual Crossing
latitude =  -1.993923  # Latitude pour Cergy, France
longitude = 30.079851  # Longitude pour Cergy, France
date = "2019-01-18"  # Date souhaitée

weather_data = get_weather_visualcrossing(api_key, latitude, longitude, date)

# Affichage des résultats
if "error" in weather_data:
    print("Erreur :", weather_data["error"])
else:
    print("Données météo :", weather_data)


Données météo : {'queryCost': 1, 'latitude': -1.993923, 'longitude': 30.079851, 'resolvedAddress': '-1.993923,30.079851', 'address': '-1.993923,30.079851', 'timezone': 'Africa/Kigali', 'tzoffset': 2.0, 'days': [{'datetime': '2019-01-18', 'datetimeEpoch': 1547762400, 'tempmax': 23.0, 'tempmin': 16.0, 'temp': 18.9, 'feelslikemax': 23.0, 'feelslikemin': 16.0, 'feelslike': 18.9, 'dew': 16.4, 'humidity': 85.8, 'precip': 0.15, 'precipprob': 100.0, 'precipcover': 8.33, 'preciptype': ['rain'], 'snow': None, 'snowdepth': None, 'windgust': None, 'windspeed': 20.5, 'winddir': 71.1, 'pressure': 1020.2, 'cloudcover': 61.9, 'visibility': 9.5, 'solarradiation': None, 'solarenergy': None, 'uvindex': None, 'sunrise': '06:03:25', 'sunriseEpoch': 1547784205, 'sunset': '18:16:32', 'sunsetEpoch': 1547828192, 'moonphase': 0.41, 'conditions': 'Rain, Partially cloudy', 'description': 'Partly cloudy throughout the day with rain in the morning and afternoon.', 'icon': 'rain', 'stations': ['HRYR', '64387099999',

In [4]:
df_test = df.head(10)

In [5]:
# Python

# Liste pour stocker les données météorologiques avec l'index
weather_records = []

# Parcourir chaque ligne du DataFrame
for index, row in df_test.iterrows():
    weather_data = get_weather_visualcrossing(api_key, row['latitude_centroid'], row['longitude_centroid'], row['date'])
    if "error" in weather_data:
        print(f"Erreur pour la ligne {index} :", weather_data["error"])
    else:
        # Extraire les données météo du jour
        day_data = weather_data['days'][0]
        # Retirer les colonnes inutiles
        day_data.pop('datetime', None)
        day_data.pop('datetimeEpoch', None)
        # Ajouter les informations de timezone
        day_data['timezone'] = weather_data.get('timezone')
        # Ajouter l'index pour aligner correctement les données plus tard
        day_data['index'] = index
        # Ajouter les données à la liste des enregistrements
        weather_records.append(day_data)

# Convertir la liste de dictionnaires en DataFrame
weather_df = pd.DataFrame(weather_records)

# Régler l'index du DataFrame météorologique sur 'index'
weather_df.set_index('index', inplace=True)

# Fusionner les données météorologiques avec df_test sur l'index
df_test = df_test.merge(weather_df, left_index=True, right_index=True, how='left')

NameError: name 'df_test' is not defined